## 问题定义

本问题与[04 LDC层流](../04_2D_LDC_Laminar/main_script.ipynb)的问题定义类似。不同点在于本问题的雷诺数更大（1000），因此不再能够视为层流，而应当视为湍流。

为了求解这个问题，本示例使用的是零方程湍流模型，即不使用微分方程，使用代数关系式把粘性系数与时均值联系起来的模型。

具体而言，具有如下的关系式：
![equation](./resource/equation.png)

其中,$l_m$为混合长度；$d$为到边界的法线距离；$d_{max}$为最大距离；$\sqrt{G}$为平均应力张量的模长（the modulus of mean rate of strain tensor）；$\nu_t$为运动粘性系数；$\mu_t$为动力粘性系数。$\nu_t = \mu_t / \rho$

其中，第三个式子，也就是混合长度通常比较难确定的，在本问题中可以使用该式计算。

### 求解目标

求解压力场、速度场以及动力粘性系数$\nu_t$

## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
import os
import warnings

from sympy import Symbol, Eq, Abs
import torch
import modulus.sym
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle

from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.eq.pdes.navier_stokes import NavierStokes
from modulus.sym.eq.pdes.turbulence_zero_eq import ZeroEquation
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 1000000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: 1000
  rec_inference_freq: 2000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: false
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ??

### 定义必要组件

#### Geo

In [4]:
height = 0.1
width = 0.1
x, y = Symbol("x"), Symbol("y")
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### PDE

分别定义零方程和NS方程

In [5]:
# class ZeroEquation(PDE):
#     """
#     Zero Equation Turbulence model

#     Parameters
#     ==========
#     nu : float
#         The kinematic viscosity of the fluid.
#     max_distance : float
#         The maximum wall distance in the flow field.
#     rho : float, Sympy Symbol/Expr, str
#         The density. If `rho` is a str then it is
#         converted to Sympy Function of form 'rho(x,y,z,t)'.
#         If 'rho' is a Sympy Symbol or Expression then this
#         is substituted into the equation. Default is 1.
#     dim : int
#         Dimension of the Zero Equation Turbulence model (2 or 3).
#         Default is 3.
#     time : bool
#         If time-dependent equations or not. Default is True.

#     Example
#     ========
#     >>> zeroEq = ZeroEquation(nu=0.1, max_distance=2.0, dim=2)
#     >>> kEp.pprint()
#       nu: sqrt((u__y + v__x)**2 + 2*u__x**2 + 2*v__y**2)
#       *Min(0.18, 0.419*normal_distance)**2 + 0.1
#     """

#     name = "ZeroEquation"

#     def __init__(
#         self, nu, max_distance, rho=1, dim=3, time=True
#     ):  # TODO add density into model
#         # set params
#         self.dim = dim
#         self.time = time

#         # model coefficients
#         self.max_distance = max_distance  # 最大长度
#         self.karman_constant = 0.419  # 3式的第一个常量
#         self.max_distance_ratio = 0.09  # 3式的第二个常量

#         # coordinates
#         x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

#         # time
#         t = Symbol("t")

#         # make input variables
#         input_variables = {"x": x, "y": y, "z": z, "t": t}
#         if self.dim == 2:
#             input_variables.pop("z")
#         if not self.time:
#             input_variables.pop("t")

#         # velocity componets
#         u = Function("u")(*input_variables)
#         v = Function("v")(*input_variables)
#         if self.dim == 3:
#             w = Function("w")(*input_variables)
#         else:
#             w = Number(0)

#         # density
#         if type(rho) is str:
#             rho = Function(rho)(*input_variables)
#         elif type(rho) in [float, int]:
#             rho = Number(rho)

#         # wall distance
#         normal_distance = Function("sdf")(*input_variables)

#         # mixing length
#         # 混合长度的计算式
#         mixing_length = Min(
#             self.karman_constant * normal_distance,
#             self.max_distance_ratio * self.max_distance,
#         )
        
#         # 计算G
#         G = (
#             2 * u.diff(x) ** 2
#             + 2 * v.diff(y) ** 2
#             + 2 * w.diff(z) ** 2
#             + (u.diff(y) + v.diff(x)) ** 2
#             + (u.diff(z) + w.diff(x)) ** 2
#             + (v.diff(z) + w.diff(y)) ** 2
#         )

#         # set equations
#         self.equations = {}
#         self.equations["nu"] = nu + rho * mixing_length**2 * sqrt(G)  # 创建nu关系式

In [6]:
# 零方程
ze = ZeroEquation(nu=1e-4, dim=2, time=False, max_distance=height / 2)
# NS方程
# nu是可学习的
ns = NavierStokes(nu=ze.equations["nu"], rho=1.0, dim=2, time=False)

#### Model

In [7]:
# 模型
# 输入为空间坐标
# 输出为u、v、p
flow_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    cfg=cfg.arch.fully_connected,
)
print(flow_net)

nodes = (
        ns.make_nodes() + ze.make_nodes() + [flow_net.make_node(name="flow_network")]
    )

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)


#### Domain

在Domain中定义约束以及训练所需的各种组件

In [8]:
# make domain
ldc_domain = Domain()

#### 边界条件

上边界为恒定的滑移边界
左右以及下边界为无滑移边界

In [9]:
# 上边界
# 水平速度为1.5
top_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 1.5, "v": 0},
    batch_size=cfg.batch_size.TopWall,
    lambda_weighting={"u": 1.0 - 20 * Abs(x), "v": 1.0},  # weight edges to be zero
    criteria=Eq(y, height / 2),
)
ldc_domain.add_constraint(top_wall, "top_wall")

# 其他边界
# 无滑移
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0, "v": 0},
    batch_size=cfg.batch_size.NoSlip,
    criteria=y < height / 2,
)
ldc_domain.add_constraint(no_slip, "no_slip")

#### 初始条件

由于求解稳态解，因此无初始条件

#### 内部PDE和ZeroEq约束

In [10]:
# interior
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.Interior,
    compute_sdf_derivatives=True,  # 以供零方程使用，计算sdf的导数，即l_m关于空间方向的导数
    lambda_weighting={
        "continuity": Symbol("sdf"),
        "momentum_x": Symbol("sdf"),
        "momentum_y": Symbol("sdf"),
    },
)
ldc_domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [11]:
class NSValidatorPlotter(ValidatorPlotter):
    "Define custom validator plotting class"

    def __call__(self, invar, true_outvar, pred_outvar):
        # only plot x,y dimensions
        # 确保仅绘制x和y，排除sdf
        invar = {k: v for k, v in invar.items() if k in ["x", "y"]}
        fs = super().__call__(invar, true_outvar, pred_outvar)
        return fs

class NSInferencerPlotter(InferencerPlotter):
    "Define custom Inferencer plotting class"

    def __call__(self, invar, pred_outvar):
        # only plot x,y dimensions
        # 确保仅绘制x和y，排除sdf
        invar = {k: v for k, v in invar.items() if k in ["x", "y"]}
        fs = super().__call__(invar, pred_outvar)
        return fs

In [12]:
# add validator
file_path = "openfoam/cavity_uniformVel_zeroEqn_refined.csv"
if os.path.exists(to_absolute_path(file_path)):
    # 加载数据
    mapping = {
        "Points:0": "x",
        "Points:1": "y",
        "U:0": "u",
        "U:1": "v",
        "p": "p",
        "d": "sdf",
        "nuT": "nu",
    }
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += -width / 2  # center OpenFoam data
    openfoam_var["y"] += -height / 2  # center OpenFoam data
    openfoam_var["nu"] += 1e-4  # effective viscosity
    openfoam_invar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["x", "y", "sdf"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "nu"]
    }
    
    # 创建validator
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=NSValidatorPlotter(),
        requires_grad=True,
    )
    ldc_domain.add_validator(openfoam_validator)

    # 创建Inferencer
    grid_inference = PointwiseInferencer(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        output_names=["u", "v", "p", "nu"],
        batch_size=1024,
        plotter=NSInferencerPlotter(),
        requires_grad=True,
    )
    ldc_domain.add_inferencer(grid_inference, "inf_data")
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

# 创建monitors
global_monitor = PointwiseMonitor(
    rec.sample_interior(4000),
    output_names=["continuity", "momentum_x", "momentum_y"],
    metrics={
        "mass_imbalance": lambda var: torch.sum(
            var["area"] * torch.abs(var["continuity"])
        ),
        "momentum_imbalance": lambda var: torch.sum(
            var["area"]
            * (torch.abs(var["momentum_x"]) + torch.abs(var["momentum_y"]))
        ),
    },
    nodes=nodes,
    requires_grad=True,
)
ldc_domain.add_monitor(global_monitor)


### 求解器以及求解

In [13]:
# 定义求解器
slv = Solver(cfg, ldc_domain)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [ ]:
slv.solve()

2024-02-18 13:06:31,600 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/05_2D_LDC_ZeroEquationTurbulence/outputs
2024-02-18 13:06:31,602 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-18 13:06:31,603 - modulus.sym.trainer - WARNING - model flow_network.0.pth not found
2024-02-18 13:06:32,969 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  9.295e-02s
2024-02-18 13:06:47,676 - modulus.sym.trainer - INFO - [step:          0] record validators time:  1.471e+01s
2024-02-18 13:06:58,326 - modulus.sym.trainer - INFO - [step:          0] record inferencers time:  1.063e+01s
2024-02-18 13:06:58,385 - modulus.sym.trainer - INFO - [step:          0] record monitor time:  4.163e-02s
2024-02-18 13:06:58,408 - modulus.sym.trainer - INFO - [step:          0] saved checkpoint to /workspace/05_2D_LDC_ZeroEquationTurbulence/outputs
2024-02-18 13:06:58,410 - modulus.sym.trainer - INFO - [step:          0] loss:  1.181e-01
20

2024-02-18 13:11:27,750 - modulus.sym.trainer - INFO - [step:       4400] loss:  7.613e-04, time/iteration:  3.819e+01 ms
2024-02-18 13:11:31,607 - modulus.sym.trainer - INFO - [step:       4500] loss:  3.139e-03, time/iteration:  3.856e+01 ms
2024-02-18 13:11:35,430 - modulus.sym.trainer - INFO - [step:       4600] loss:  2.011e-03, time/iteration:  3.821e+01 ms
2024-02-18 13:11:39,308 - modulus.sym.trainer - INFO - [step:       4700] loss:  2.244e-03, time/iteration:  3.877e+01 ms
2024-02-18 13:11:43,256 - modulus.sym.trainer - INFO - [step:       4800] loss:  6.169e-04, time/iteration:  3.944e+01 ms
2024-02-18 13:11:47,205 - modulus.sym.trainer - INFO - [step:       4900] loss:  7.676e-04, time/iteration:  3.948e+01 ms
2024-02-18 13:12:06,756 - modulus.sym.trainer - INFO - [step:       5000] record validators time:  1.466e+01s
2024-02-18 13:12:06,822 - modulus.sym.trainer - INFO - [step:       5000] record monitor time:  6.402e-02s
2024-02-18 13:12:06,867 - modulus.sym.trainer - INF

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果

![v](./outputs/validators/validator_v.png)

![u](./outputs/validators/validator_u.png)

![nu](./outputs/validators/validator_nu.png)


#### 推理结果

![p](./outputs/inferencers/inf_data_p.png)